# Orquestrador de Chatbots - Etapa 1
-----------------------------------
## Extração e Separação dos Dados

1. Extrair as perguntas dos arquivos JSON de Skills dos bots.
2. Separar os dados em massa para treino/teste (rotulada) dos classificadores e de validação (rotulada e não rotulada), para posterior avaliação do desempenho dos classificadores gerados.
3. Incluir na massa de validação uma série de perguntas que não pertencem a nenhum bot, simulando um ambiente real de produção.
4. Salvar os três conjuntos de dados em arquivos CSV.

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Bibliotecas utilizadas

In [27]:
# https://pypi.org/project/pandas/
import pandas as pd

# https://numpy.org/
import numpy as np

# https://docs.python.org/3/library/csv.html
import csv

# https://www.nltk.org/api/nltk.tokenize.html
from nltk.tokenize import RegexpTokenizer

# https://docs.python.org/3/library/re.html
import re

# https://docs.python.org/3/library/unicodedata.html
from unicodedata import normalize

# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
from sklearn.feature_extraction.text import CountVectorizer

# https://scikit-learn.org/stable/modules/model_evaluation.html
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.model_selection import train_test_split

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
from sklearn.model_selection import StratifiedKFold

# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression

# https://docs.python.org/3/library/statistics.html
from statistics import mean 

import json
import codecs
import os
import glob

In [28]:
import orquestrador_funcoes_gerais as ofg

## Funções

In [29]:
# Inserir funções específicas do notebook aqui.

## Configurações

In [30]:
cfg = ofg.carregar_configuracoes()

Conferir as configurações antes de prosseguir
nome_arquivo_configuracao: config.json
------------------------------------------------------------------------------------------------------------------------
bots: [{'bot_id': 1, 'nome': 'Alistamento Militar', 'arquivo': 'skill-alistamento-militar.json'}, {'bot_id': 2, 'nome': 'COVID', 'arquivo': 'skill-covid.json'}, {'bot_id': 3, 'nome': 'Login Único', 'arquivo': 'skill-login-unico.json'}, {'bot_id': 4, 'nome': 'IRPF 2020', 'arquivo': 'skill-perguntao-irpf-2020.json'}, {'bot_id': 5, 'nome': 'PGMEI', 'arquivo': 'skill-pgmei.json'}, {'bot_id': 6, 'nome': 'Selo Turismo Responsável', 'arquivo': 'skill-poc-selo-turismo-responsavel.json'}, {'bot_id': 7, 'nome': 'Cadastur', 'arquivo': 'skill-cadastur.json'}, {'bot_id': 8, 'nome': 'Tuberculose', 'arquivo': 'skill-tuberculose.json'}]
------------------------------------------------------------------------------------------------------------------------
diretorio_skills: skills
-------------------

In [45]:
df[df.isna().any(axis=1)]

,bot_id,pergunta


In [31]:
bots=cfg['bots']

##  Extrai Perguntas das Skills dos JSONs

1. Lê os JSONs de skills dos bots
2. Extrai deles as perguntas ("exemplos")
3. Converte as  perguntas para o formato CSV
4. Salva o CSV para uso posterior.

In [32]:
arquivo_todas = os.path.join(os.getcwd(),  cfg['diretorio_dados'], cfg['arquivo_todas']) 

with codecs.open(arquivo_todas, 'w', 'utf-8') as csvfile:
    csvfile.write('bot_id,pergunta\n')
    total_geral = 0
    for bot in bots:   
        arquivo_skill = os.path.join(os.getcwd(),  cfg['diretorio_skills'], bot['arquivo']) 
        skills = json.loads(open(arquivo_skill, encoding='utf-8').read())
        total_bot = 0
        for intent in skills['intents']:
            for example in intent['examples']:
                csvfile.write(str(bot['bot_id']) + ',' + example['text'].replace(',',' ') + '\n')
                total_bot += 1
                total_geral += 1                
        print(bot['bot_id'],'-',bot['nome'],'- Arquivo:',bot['arquivo'],'- Total perguntas extraídas:',total_bot)
    csvfile.close()
    
print('\nTotal de Perguntas Extraídas:',total_geral)
print('Perguntas gravadas no arquivo:', arquivo_todas)

1 - Alistamento Militar - Arquivo: skill-alistamento-militar.json - Total perguntas extraídas: 365
2 - COVID - Arquivo: skill-covid.json - Total perguntas extraídas: 178
3 - Login Único - Arquivo: skill-login-unico.json - Total perguntas extraídas: 449
4 - IRPF 2020 - Arquivo: skill-perguntao-irpf-2020.json - Total perguntas extraídas: 1850
5 - PGMEI - Arquivo: skill-pgmei.json - Total perguntas extraídas: 111
6 - Selo Turismo Responsável - Arquivo: skill-poc-selo-turismo-responsavel.json - Total perguntas extraídas: 599
7 - Cadastur - Arquivo: skill-cadastur.json - Total perguntas extraídas: 294
8 - Tuberculose - Arquivo: skill-tuberculose.json - Total perguntas extraídas: 232

Total de Perguntas Extraídas: 4078
Perguntas gravadas no arquivo: E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\dados\bots_perguntas_todas.csv


## Carrega os dados extraídos

Carrega os dados do arquivo CSV preparado anteriormente.

In [33]:
df = pd.read_csv(arquivo_todas, index_col=None, engine='python', sep =',', encoding="utf-8")
print('Total de registros carregados:',len(df))

# Exibe uma amostra dos dados.
df.tail(-1)

Total de registros carregados: 4078


,bot_id,pergunta
1,1,como proceder para adiar a incorporação no ali...
2,1,cursando faculdade na area da saude na época d...
3,1,Estou cursando a faculdade de Medicina Farmác...
4,1,estudantes da area de saúde têm condições espe...
5,1,Na época do alistamento militar o que deve faz...
...,...,...
4073,8,Tuberculose Urinaria
4074,8,tuberculose urinaria é o que
4075,8,Tuberculose Urinaria - Como tratar
4076,8,Tuberculose Urinaria - Diagnostico


## Separa dados de treino/teste e de validação

In [34]:
# Os registros deverão ser embaralhados, mas a distribuição das classes deve permanecer a mesma para os dois conjuntos de dados.
df_treino_teste, df_validacao = train_test_split(df, test_size=cfg['validacao_size'], 
                                                 stratify=df['bot_id'], shuffle=True, random_state=cfg['random_state'])

In [35]:
print('Total:',len(df),'- Treino/Teste:',len(df_treino_teste),'- Validação:',len(df_validacao))

Total: 4078 - Treino/Teste: 2650 - Validação: 1428


In [36]:
# Distribuição das classes nos dados fornecidos.
df.groupby('bot_id').count()

,pergunta
bot_id,
1,365
2,178
3,449
4,1850
5,111
6,599
7,294
8,232


In [37]:
# Distribuição das classes nos dados de treino e teste.
df_treino_teste.groupby('bot_id').count()

,pergunta
bot_id,
1,237
2,116
3,292
4,1202
5,72
6,389
7,191
8,151


In [38]:
# Distribuição das classes nos dados de validação.
df_validacao.groupby('bot_id').count()

,pergunta
bot_id,
1,128
2,62
3,157
4,648
5,39
6,210
7,103
8,81


In [39]:
# Salva dados de treino e teste
arquivo_treino_testes = os.path.join(os.getcwd(),  cfg['diretorio_dados'], cfg['arquivo_treino_testes'])
df_treino_teste.to_csv(arquivo_treino_testes, index=False)
print('Dados de Treino e Teste salvos em', arquivo_treino_testes)

# Salva dados para validação, sem as "perguntas zero", para conferência da performance dos classificadores
arquivo_validacao = os.path.join(os.getcwd(),  cfg['diretorio_dados'], cfg['arquivo_validacao'])
df_validacao.to_csv(arquivo_validacao, index=False)
print('Dados de Validação (SEM perguntas 0) salvos em', arquivo_validacao)

Dados de Treino e Teste salvos em E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\dados\treino_testes.csv
Dados de Validação (SEM perguntas 0) salvos em E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\dados\validacao.csv


## Inclui na validação perguntas não relacionadas a nenhum dos bots

Terão que ser pensadas estratégias para lidar com perguntas que não estão no contexto de nenhum dos bots do escopo.
Essas perguntas "Zero" simulam essa situação, muito comum num ambiente real de produção.

In [40]:
print('Tamanho do Arquivo de Validação:',len(df_validacao),'perguntas')

# Carrega as perguntas '0'
df0 = pd.read_csv(os.path.join(os.getcwd(),  cfg['diretorio_dados'], cfg['arquivo_perguntas_zero']), 
                  index_col=None, engine='python', sep ='|', encoding="utf-8")
print(len(df0),'perguntas "Zero" carregadas')

# Acrescenta o identificador 0
df0['bot_id'] = 0

# Adiciona as perguntas ao arquivo de validação
df_validacao = df_validacao.append(df0,ignore_index=True)

# Embaralha e reindexa o arquivo de validação
df_validacao = df_validacao.sample(frac = 1)
df_validacao = df_validacao.reset_index(drop=True)

print('Novo Tamanho do Arquivo de Validação:',len(df_validacao),'perguntas')
print('Perguntas "Zero" são', round(len(df0)/len(df_validacao),4)*100, '% do total')
    

Tamanho do Arquivo de Validação: 1428 perguntas
255 perguntas "Zero" carregadas
Novo Tamanho do Arquivo de Validação: 1683 perguntas
Perguntas "Zero" são 15.15 % do total


In [41]:
df_validacao.groupby('bot_id').count()

,pergunta
bot_id,
0,255
1,128
2,62
3,157
4,648
5,39
6,210
7,103
8,81


In [42]:
# Salva dados para validação, COM as "perguntas zero"
arquivo_validacao_com_zero = os.path.join(os.getcwd(),  cfg['diretorio_dados'], cfg['arquivo_validacao_com_zero'])
df_validacao.to_csv(arquivo_validacao_com_zero, index=False)
print('Dados de Validação (COM perguntas 0) salvos em', arquivo_validacao_com_zero)

Dados de Validação (COM perguntas 0) salvos em E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\dados\validacao_com_0.csv


In [44]:
print("Fim da etapa 1!")

Fim da etapa 1!
